Copyright 2017 Google LLC.

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

# MusicVAE: A Hierarchical Latent Vector Model for Learning Long-Term Structure in Music.
### ___Adam Roberts, Jesse Engel, Colin Raffel, Curtis Hawthorne, and Douglas Eck___

[MusicVAE](https://g.co/magenta/music-vae) learns a latent space of musical scores, providing different modes
of interactive musical creation, including:

* Random sampling from the prior distribution.
* Interpolation between existing sequences.
* Manipulation of existing sequences via attribute vectors.

Examples of these interactions can be generated below, and selections can be heard in our
[YouTube playlist](https://www.youtube.com/playlist?list=PLBUMAYA6kvGU8Cgqh709o5SUvo-zHGTxr).

For short sequences (e.g., 2-bar "loops"), we use a bidirectional LSTM encoder
and LSTM decoder. For longer sequences, we use a novel hierarchical LSTM
decoder, which helps the model learn longer-term structures.

We also model the interdependencies between instruments by training multiple
decoders on the lowest-level embeddings of the hierarchical decoder.

For additional details, check out our [blog post](https://g.co/magenta/music-vae) and [paper](https://goo.gl/magenta/musicvae-paper).
___

This colab notebook is self-contained and should run natively on google cloud. The [code](https://github.com/tensorflow/magenta/tree/master/magenta/models/music_vae) and [checkpoints](http://download.magenta.tensorflow.org/models/music_vae/checkpoints.tar.gz) can be downloaded separately and run locally, which is required if you want to train your own model.

# Basic Instructions

1. Double click on the hidden cells to make them visible, or select "View > Expand Sections" in the menu at the top.
2. Hover over the "`[ ]`" in the top-left corner of each cell and click on the "Play" button to run it, in order.
3. Listen to the generated samples.
4. Make it your own: copy the notebook, modify the code, train your own models, upload your own MIDI, etc.!

# Environment Setup
Includes package installation for sequence synthesis. Will take a few minutes.


In [ ]:
#@title Setup Environment
#@test {"output": "ignore"}

import glob

BASE_DIR = "gs://download.magenta.tensorflow.org/models/music_vae/colab2"

print('Installing dependencies...')
!apt-get update -qq && apt-get install -qq libfluidsynth1 fluid-soundfont-gm build-essential libasound2-dev libjack-dev
!pip install -q pyfluidsynth
!pip install -qU magenta

# Hack to allow python to pick up the newly-installed fluidsynth lib.
# This is only needed for the hosted Colab environment.
import ctypes.util
orig_ctypes_util_find_library = ctypes.util.find_library
def proxy_find_library(lib):
  if lib == 'fluidsynth':
    return 'libfluidsynth.so.1'
  else:
    return orig_ctypes_util_find_library(lib)
ctypes.util.find_library = proxy_find_library


print('Importing libraries and defining some helper functions...')
from google.colab import files
import magenta.music as mm
from magenta.models.music_vae import configs
from magenta.models.music_vae.trained_model import TrainedModel
import numpy as np
import os
import tensorflow.compat.v1 as tf

tf.disable_v2_behavior()
# tf.enable_eager_execution()

# Necessary until pyfluidsynth is updated (>1.2.5).
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

def play(note_sequence):
  mm.play_sequence(note_sequence, synth=mm.fluidsynth)

def interpolate(model, start_seq, end_seq, num_steps, max_length=32,
                assert_same_length=True, temperature=0.5,
                individual_duration=4.0):
  """Interpolates between a start and end sequence."""
  note_sequences = model.interpolate(
      start_seq, end_seq,num_steps=num_steps, length=max_length,
      temperature=temperature,
      assert_same_length=assert_same_length)

  print('Start Seq Reconstruction')
  play(note_sequences[0])
  print('End Seq Reconstruction')
  play(note_sequences[-1])
  print('Mean Sequence')
  play(note_sequences[num_steps // 2])
  print('Start -> End Interpolation')
  interp_seq = mm.sequences_lib.concatenate_sequences(
      note_sequences, [individual_duration] * len(note_sequences))
  play(interp_seq)
  mm.plot_sequence(interp_seq)
  return interp_seq if num_steps > 3 else note_sequences[num_steps // 2]

def download(note_sequence, filename):
  mm.sequence_proto_to_midi_file(note_sequence, filename)
  files.download(filename)

print('Done')

Installing dependencies...
Selecting previously unselected package fluid-soundfont-gm.
(Reading database ... 155222 files and directories currently installed.)
Preparing to unpack .../fluid-soundfont-gm_3.1-5.1_all.deb ...
Unpacking fluid-soundfont-gm (3.1-5.1) ...
Selecting previously unselected package libfluidsynth1:amd64.
Preparing to unpack .../libfluidsynth1_1.1.9-1_amd64.deb ...
Unpacking libfluidsynth1:amd64 (1.1.9-1) ...
Setting up fluid-soundfont-gm (3.1-5.1) ...
Setting up libfluidsynth1:amd64 (1.1.9-1) ...
Processing triggers for libc-bin (2.27-3ubuntu1.3) ...
/sbin/ldconfig.real: /usr/local/lib/python3.7/dist-packages/ideep4py/lib/libmkldnn.so.0 is not a symbolic link

     |████████████████████████████████| 1.4 MB 13.7 MB/s 
     |████████████████████████████████| 1.4 MB 60.0 MB/s 
     |████████████████████████████████| 3.6 MB 54.7 MB/s 
     |████████████████████████████████| 254 kB 61.2 MB/s 
     |████████████████████████████████| 204 kB 71.5 MB/s 
     |█████████████

/usr/local/lib/python3.7/dist-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
/usr/local/lib/python3.7/dist-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit


Instructions for updating:
non-resource variables are not supported in the long term
Done


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/Thesis/Code/Magenta/magenta/

/content/drive/MyDrive/University of Alberta/Thesis/Code/Magenta/magenta


In [ ]:
import datetime
%load_ext tensorboard

# 2-Bar Melody Model

The pre-trained model consists of a single-layer bidirectional LSTM encoder with 2048 nodes in each direction, a 3-layer LSTM decoder with 2048 nodes in each layer, and Z with 512 dimensions. The model was given 0 free bits, and had its beta valued annealed at an exponential rate of 0.99999 from 0 to 0.43 over 200k steps. It was trained with scheduled sampling with an inverse sigmoid schedule and a rate of 1000. The final accuracy is 0.95 and KL divergence is 58 bits.

In [ ]:
#@title Load the pre-trained model.
mel_2bar_config = configs.CONFIG_MAP['cat-mel_2bar_big']
mel_2bar = TrainedModel(mel_2bar_config, batch_size=4, checkpoint_dir_or_path=BASE_DIR + '/checkpoints/mel_2bar_big.ckpt')

INFO:tensorflow:Building MusicVAE model with BidirectionalLstmEncoder, CategoricalLstmDecoder, and hparams:
{'max_seq_len': 32, 'z_size': 512, 'free_bits': 0, 'max_beta': 0.5, 'beta_rate': 0.99999, 'batch_size': 4, 'grad_clip': 1.0, 'clip_mode': 'global_norm', 'grad_norm_clip_to_zero': 10000, 'learning_rate': 0.001, 'decay_rate': 0.9999, 'min_learning_rate': 1e-05, 'conditional': True, 'dec_rnn_size': [2048, 2048, 2048], 'enc_rnn_size': [2048], 'dropout_keep_prob': 1.0, 'sampling_schedule': 'inverse_sigmoid', 'sampling_rate': 1000, 'use_cudnn': False, 'residual_encoder': False, 'residual_decoder': False, 'control_preprocessing_rnn_size': [256]}
INFO:tensorflow:
Encoder Cells (bidirectional):
  units: [2048]

INFO:tensorflow:
Decoder Cells:
  units: [2048, 2048, 2048]

Instructions for updating:
Use `tf.cast` instead.


/usr/local/lib/python3.7/dist-packages/magenta/models/music_vae/lstm_utils.py:99: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  name=name),
/usr/local/lib/python3.7/dist-packages/keras/legacy_tf_layers/core.py:255: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  return layer.apply(inputs)
/usr/local/lib/python3.7/dist-packages/magenta/contrib/rnn.py:751: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  self._names["W"], [input_size + self._num_units, self._num_units * 4])
/usr/local/lib/python3.7/dist-packages/magenta/contrib/rnn.py:754: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  initializer=tf.constant_initializer(0.0))


Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell))`, which is equivalent to this API
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Do not call `graph_parents`.


/usr/local/lib/python3.7/dist-packages/magenta/models/music_vae/base_model.py:199: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  kernel_initializer=tf.random_normal_initializer(stddev=0.001))
/usr/local/lib/python3.7/dist-packages/magenta/models/music_vae/base_model.py:205: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  kernel_initializer=tf.random_normal_initializer(stddev=0.001))


INFO:tensorflow:Restoring parameters from gs://download.magenta.tensorflow.org/models/music_vae/colab2/checkpoints/mel_2bar_big.ckpt


## Generate Samples

In [ ]:
#@title Generate 4 samples from the prior.
temperature = 0.5 #@param {type:"slider", min:0.1, max:1.5, step:0.1}
mel_2_samples = mel_2bar.sample(n=4, length=32, temperature=temperature)
for ns in mel_2_samples:
  play(ns)

In [ ]:
type(mel_2_samples)

list

In [ ]:
type(mel_2_samples[0])

note_seq.protobuf.music_pb2.NoteSequence

In [ ]:
#@title Optionally download samples.
for i, ns in enumerate(mel_2_samples):
  download(ns, 'mel_2bar_sample_%d.mid' % i)

# Fine-tuned 2-Bar Melody Model


In [ ]:
# !tar -cvf ./data/checkpoints/train-10-26-03-finetune-1000.ckpt ./data/tmp/train-10-26-03-finetune/model.ckpt-1000.*

./data/tmp/train-10-26-03-finetune/model.ckpt-1000.data-00000-of-00001
./data/tmp/train-10-26-03-finetune/model.ckpt-1000.index
./data/tmp/train-10-26-03-finetune/model.ckpt-1000.meta


In [ ]:
mel_2bar_config = configs.CONFIG_MAP['cat-mel_2bar_big']
mel_2bar_finetune = TrainedModel(mel_2bar_config, batch_size=4, checkpoint_dir_or_path = './data/checkpoints/train-10-26-03-finetune-1000.ckpt')

INFO:tensorflow:Building MusicVAE model with BidirectionalLstmEncoder, CategoricalLstmDecoder, and hparams:
{'max_seq_len': 32, 'z_size': 512, 'free_bits': 0, 'max_beta': 0.5, 'beta_rate': 0.99999, 'batch_size': 4, 'grad_clip': 1.0, 'clip_mode': 'global_norm', 'grad_norm_clip_to_zero': 10000, 'learning_rate': 0.001, 'decay_rate': 0.9999, 'min_learning_rate': 1e-05, 'conditional': True, 'dec_rnn_size': [2048, 2048, 2048], 'enc_rnn_size': [2048], 'dropout_keep_prob': 1.0, 'sampling_schedule': 'inverse_sigmoid', 'sampling_rate': 1000, 'use_cudnn': False, 'residual_encoder': False, 'residual_decoder': False, 'control_preprocessing_rnn_size': [256]}
INFO:tensorflow:
Encoder Cells (bidirectional):
  units: [2048]

INFO:tensorflow:
Decoder Cells:
  units: [2048, 2048, 2048]



/usr/local/lib/python3.7/dist-packages/magenta/models/music_vae/lstm_utils.py:99: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  name=name),
/usr/local/lib/python3.7/dist-packages/keras/legacy_tf_layers/core.py:255: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  return layer.apply(inputs)
/usr/local/lib/python3.7/dist-packages/magenta/contrib/rnn.py:751: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  self._names["W"], [input_size + self._num_units, self._num_units * 4])
/usr/local/lib/python3.7/dist-packages/magenta/contrib/rnn.py:754: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  initializer=tf.constant_initializer(0.0))
/usr/local/lib/python3.

INFO:tensorflow:Unbundling checkpoint.
INFO:tensorflow:Restoring parameters from /tmp/tmp35birp51/./data/tmp/train-10-26-03-finetune/model.ckpt-1000


In [ ]:
!python magenta/models/music_vae/music_vae_train_checkpoint.py \
--config=cat-mel_2bar_big \
--run_dir=./data/tmp/Horror_eval/ \
--mode=eval \
--examples_path=./data/tfrecord/Horror.tfrecord \
--log=DEBUG \
--hparams=batch_size=15 \
--cache_dataset=False

/usr/local/lib/python3.7/dist-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
/usr/local/lib/python3.7/dist-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
Instructions for updating:
non-resource variables are not supported in the long term
INFO:tensorflow:Counting examples in ./data/tfrecord/Horror.tfrecord.
I1119 18:39:08.710342 140547612485504 data.py:1682] Counting example

In [ ]:
%tensorboard --logdir ./data/tmp/Horror_eval/eval/

In [ ]:
!python magenta/models/music_vae/music_vae_train_checkpoint.py \
--config=cat-mel_2bar_big \
--run_dir=./data/tmp/Video_game_eval/ \
--mode=eval \
--examples_path=./data/tfrecord/Video_game.tfrecord \
--log=DEBUG \
--hparams=batch_size=3 \
--cache_dataset=False

python3: can't open file 'magenta/models/music_vae/music_vae_train_checkpoint.py': [Errno 2] No such file or directory


In [ ]:
%tensorboard --logdir ./data/tmp/Video_game_eval/eval/

In [ ]:
!python magenta/models/music_vae/music_vae_train_checkpoint.py \
--config=cat-mel_2bar_big \
--run_dir=./data/tmp/Persian2_eval \
--mode=eval \
--examples_path=./data/tfrecord/Persian2_eval.tfrecord \
--log=DEBUG \
--hparams=batch_size=2 \
--cache_dataset=False

/usr/local/lib/python3.7/dist-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
/usr/local/lib/python3.7/dist-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
Instructions for updating:
non-resource variables are not supported in the long term
INFO:tensorflow:Counting examples in ./data/tfrecord/Persian2_eval.tfrecord.
I1126 19:02:06.715785 140408579069824 data.py:1682] Counting 

In [ ]:
!kill 2391

In [ ]:
%tensorboard --logdir ./data/tmp/Persian2_eval/eval/

In [ ]:
!python magenta/models/music_vae/music_vae_train_checkpoint.py \
--config=cat-mel_2bar_big \
--run_dir=./data/tmp/Pop_eval/ \
--mode=eval \
--examples_path=./data/tfrecord/Pop.tfrecord \
--log=DEBUG \
--hparams=batch_size=3 \
--cache_dataset=False

/usr/local/lib/python3.7/dist-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
/usr/local/lib/python3.7/dist-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
Instructions for updating:
non-resource variables are not supported in the long term
INFO:tensorflow:Counting examples in ./data/tfrecord/Pop.tfrecord.
I1119 20:16:29.130543 140218699954048 data.py:1682] Counting examples i

In [ ]:
%tensorboard --logdir ./data/tmp/Pop_eval/eval/

# Data Setup (Horror)

In [ ]:
tf.enable_eager_execution()

In [ ]:
from magenta.models.music_vae import data
from magenta.models.music_vae import data

In [ ]:
data_path = './data/tfrecord/Bo_Burnham_train.tfrecord'
tf_file_reader = tf.data.TFRecordDataset
file_reader = tf.python_io.tf_record_iterator

In [ ]:
mel_2bar_config = configs.update_config(mel_2bar_config, dict(eval_examples_path=data_path))

In [ ]:
mel_2bar_config.hparams.batch_size = 1

In [ ]:
dataset = data.get_dataset(
    mel_2bar_config,
    tf_file_reader=tf_file_reader,
    is_training=False,
    cache_dataset=False)

INFO:tensorflow:Reading examples from file: ./data/tfrecord/Bo_Burnham_train.tfrecord


In [ ]:
dataset = dataset.take(-1)

In [ ]:
dataset

<TakeDataset shapes: ((1, ?, 90), (1, ?, 90), (1, ?, 0), (1,)), types: (tf.bool, tf.bool, tf.bool, tf.int32)>

In [ ]:
batch_size = mel_2bar_config.hparams.batch_size
iterator = tf.data.make_one_shot_iterator(dataset)

In [ ]:
input_seqs, output_seqs, control_seqs, sequence_lengths = [], [], [], []

In [ ]:
for i, o, c, sl in iterator:
  input_seqs.append(mel_2bar_config.data_converter.from_tensors(i)[0])
  # input_seqs.append(i)
  sequence_lengths.append(sl)

In [ ]:
len(input_seqs)

39

In [ ]:
input_seqs_new = list()
for i in input_seqs:
  try:
    a, b, c = mel_2bar.encode([i])
  except:
    print("invalid")
  else:
    input_seqs_new.append(i)

invalid
invalid
invalid


# Reconstruction

In [ ]:
# plain model
z1, mu1, sigma1 = mel_2bar.encode(input_seqs_new)
z1 = [z1[i] for i in range(z1.shape[0])]
output_seqs1 = mel_2bar.decode(z1, length=32)

In [ ]:
# fine-tuned model
z2, mu2, sigma2 = mel_2bar_finetune.encode(input_seqs_new)
z2 = [z2[i] for i in range(z2.shape[0])]
output_seqs2 = mel_2bar_finetune.decode(z2, length=32)

# Comparison

In [ ]:
for i in range(0, len(input_seqs_new)):
  print("Sample {}: ".format(i+1))
  play(input_seqs_new[i])
  print("Plain: ")
  play(output_seqs1[i])
  print("Fine-tune: ")
  play(output_seqs2[i])
  print("----------------------------------")


Sample 1: 


Plain: 


Fine-tune: 


----------------------------------
Sample 2: 


Plain: 


...

Fine-tune: 


----------------------------------
Sample 36: 


Plain: 


Fine-tune: 


----------------------------------


# Sampling

## plain

In [ ]:
#@title Generate 4 samples from the prior.
temperature = 1 #@param {type:"slider", min:0.1, max:1.5, step:0.1}
mel_2_samples = mel_2bar.sample(n=4, length=128, temperature=temperature)
for ns in mel_2_samples:
  play(ns)

## finetune

In [ ]:
#@title Generate 4 samples from the prior.
temperature = 1 #@param {type:"slider", min:0.1, max:1.5, step:0.1}
mel_2_samples_finetune = mel_2bar_finetune.sample(n=4, length=128, temperature=temperature)
for ns in mel_2_samples_finetune:
  play(ns)